# Install and Import phidata
Install the phidata library and import necessary modules for working with agents, tools, knowledge, and storage components.

In [ ]:
! pip install numpy==1.26.4 ipykernel phidata openai ipywidgets duckduckgo-search yfinance crawl4ai lancedb sentence-transformers torch pypdf chromadb duckdb


### Explanation of the Code

The code performs the following steps:

1. **Importing Required Modules**:
    - `os`: Provides functions to interact with the operating system.
    - `load_dotenv` from `dotenv`: Loads environment variables from a `.env` file into the system environment.

2. **Loading Environment Variables**:
    - The `load_dotenv()` function is called to load environment variables from a `.env` file.

3. **Clearing Conflicting Environment Variables**:
    - A list of environment variables (`env_vars_to_clear`) is defined, which includes `OPENAI_API_KEY`, `OPENAI_BASE_URL`, and `OPENAI_API_BASE`.
    - For each variable in the list, the code checks if it exists in the environment. If it does, it prints a warning message and deletes the variable from the environment.

4. **Setting New Environment Variables**:
    - The `OPENAI_API_KEY` is set to the value of the `OPEN_ROUTER_KEY` environment variable.
    - Both `OPENAI_API_BASE` and `OPENAI_BASE_URL` are set to `'https://openrouter.ai/api/v1'`.

5. **Commented-Out Code**:
    - There is an alternative block of code (commented out) that clears the same environment variables and sets `OPENAI_API_KEY` to the value of the `OPEN_AI_KEY` environment variable instead.

### Purpose:
This code ensures that conflicting environment variables are removed and sets up the required environment variables for interacting with the OpenRouter API. It provides flexibility to switch between different API keys and base URLs by modifying the `.env` file or uncommenting the alternative block of code.


In [ ]:

import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv()


env_vars_to_clear = ['OPENAI_API_KEY', 'OPENAI_BASE_URL', 'OPENAI_API_BASE']
for var in env_vars_to_clear:
    if os.getenv(var):
        print(f"⚠️  Removing conflicting {var}")
        del os.environ[var]


os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_ROUTER_KEY")
os.environ['OPENAI_API_BASE'] = 'https://openrouter.ai/api/v1'
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'



# env_vars_to_clear = ['OPENAI_API_KEY', 'OPENAI_BASE_URL', 'OPENAI_API_BASE']
# for var in env_vars_to_clear:
#     if os.getenv(var):
#         print(f"⚠️  Removing conflicting {var}")
#         del os.environ[var]
# os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_KEY")




### Explanation of the Code

The code in the next cell performs the following steps:

1. **Importing Required Modules**:
    - `phi.agent.Agent`: Used to create an agent that can perform reasoning tasks.
    - `phi.tools.Tool`: Provides tools that can be used by the agent.
    - `phi.model.openai.OpenAIChat`: Represents the OpenAI GPT model used for generating responses.
    - `phi.utils.pprint.pprint_run_response`: A utility function to pretty-print the agent's response.

### Purpose:
This code sets up the necessary imports to create and interact with agents that utilize OpenAI's GPT models for reasoning and task execution. It also includes tools and utilities for enhancing the agent's functionality and formatting its output.

In [ ]:
from phi.agent import Agent
from phi.tools import Tool
from phi.model.openai import OpenAIChat
from phi.utils.pprint import pprint_run_response

In [ ]:
import httpx
from pathlib import Path
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.csv_tools import CsvTools
from phi.tools.python import PythonTools

# Download IMDB CSV file
# url = "https://phidata-public.s3.amazonaws.com/demo_data/IMDB-Movie-Data.csv"
# response = httpx.get(url)

imdb_csv = "imdb.csv"



# Create agent with CSV and Python tools
knowledge_graph_agent = Agent(
    name="IMDB Knowledge Graph Agent",
    model=OpenAIChat(id="gpt-4o"),
    tools=[
        CsvTools(csvs=[imdb_csv]),
        PythonTools(pip_install=True, save_and_run=True)
    ],
    instructions=[
        "You are an expert at analyzing movie data and creating knowledge graphs",
        "First, explore the CSV file to understand its structure and columns",
        "Then create a comprehensive knowledge graph showing relationships between movies, directors, actors, genres, and ratings",
        "Use networkx and matplotlib/plotly for visualization",
        "Always show the graph structure and save visualizations"
    ],
    markdown=True,
    show_tool_calls=True,
)

# Ask the agent to create a knowledge graph
knowledge_graph_agent.print_response("""
                                     use  only 200 records
Please analyze the IMDB movie dataset and create a knowledge graph that shows:

1. First, examine the CSV file structure and show me the columns and sample data
2. Create a knowledge graph that represents:
   
   - Directors as nodes  
   - Genres as nodes
   - actors as nodes
   - Movies as nodes
   
   - Edge weights based on movie ratings or revenue
3. Visualize the knowledge graph using networkx and matplotlib
4. Provide insights about the most connected nodes (popular directors, common genres, etc.)
5. Save the visualization as an image file

Use appropriate graph layout algorithms to make the visualization clear and informative.
""", stream=True)